In [ ]:
!pip install spacy==3.1.0
!python -m spacy download pt_core_news_lg # Após instalação, reiniciar o ambiente do colab.

In [29]:
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, precision_recall_fscore_support)
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report

In [30]:
import spacy

spc = spacy.load("pt_core_news_sm")

In [ ]:
import nltk
nltk.download('stopwords')
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [33]:
default_dir = "/content/drive/MyDrive/UFS/UFS-2020.2/tcc2/data/"
data_tcc_pos_neg = default_dir+'labeled_data/dataset_label_pos_neg.csv'
data_neg_emotions = default_dir+'labeled_data/dataset_neg_emotions.csv'


In [ ]:
data = pd.read_csv(data_tcc_pos_neg)
data['type'] = 'with_theme-' + data['sentiment']
data = data.dropna()
data.head()

In [35]:
def lemma_words(doc):
  spc_letras = spc(str(doc))
  return (token.lemma_ if token.pos_ == 'VERB' else str(token) for token in spc_letras)

In [ ]:
def create_splits(data):
    test_validation_size = int(0.20*data.shape[0])
    train, test = train_test_split(data, test_size=test_validation_size, random_state=42, stratify=data['type'])
    return train, test

  
train, test = create_splits(data)
print('Training samples:  ', train.shape[0])
print('Test samples:      ', test.shape[0])

In [37]:
vectorizer = TfidfVectorizer(
stop_words=nltk.corpus.stopwords.words('portuguese'), 
    analyzer=lemma_words,
    min_df=0.0001, 
    max_df=0.8,
    smooth_idf=True
)

In [38]:
X_train = vectorizer.fit_transform(train['clean_text'].values.astype('U'))
X_test = vectorizer.transform(test['clean_text'].values.astype('U'))

In [39]:
labels = {
    'Positivo' : 0,
    'Negativo' : 1,
}

# encoding = {
#     'tristeza': 0,
#     'medo': 1,
#     'raiva': 2,
#     'desprezo' : 3,
# }

y_train = train['sentiment'].map(labels).values
y_test = test['sentiment'].map(labels).values

In [ ]:
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(X_train, y_train)

In [41]:
prediction_linear = classifier_linear.predict(X_test)

In [ ]:
print(classification_report(y_test, prediction_linear))


In [ ]:
ax = sns.heatmap(confusion_matrix(y_test, prediction_linear), cmap='Greens_r', annot=True, fmt='d')
_ = ax.set(xlabel='Previsto', ylabel='Correto', title='SVM')

In [ ]:
#Calculando a acurácia
acc = accuracy_score(prediction_linear, y_test)
precision_recall_fscore = precision_recall_fscore_support(prediction_linear, y_test, average='weighted')
print(f'Acurácia: {acc}')
print(f'\Result: {precision_recall_fscore}')